In [19]:
# V1
# upto 20190330
# calculate decile first
# then merge email
# then remove the 18+


# In other words: all the same, except for the time range
import pandas as pd
import numpy as np
import os
import datetime
import logging
import hashlib
import gc
import glob
logging.basicConfig(filename='V1_BL_CRM_LR_Quarterly_20190417_upto20190330_Decile_MergeEmail_Remove18+.log', level=logging.INFO)
logging.info('Started')

samplerows = None

lastdate = datetime.date(2019,3,30) # Recent Saturday
active_Sunday = str(lastdate-datetime.timedelta(days=52*7-1))
lapsed_Sunday = str(lastdate-datetime.timedelta(days=52*7*1.5-1))
Beginning_18_months_ago=str(lastdate-datetime.timedelta(days=52*7*1.5-1))
lastdate=str(lastdate)
print("Lapsed Start on: "+lapsed_Sunday) #>=
print("Active Start on: "+active_Sunday) #>=
print("Store Allocation Starting on: "+Beginning_18_months_ago) #>=
print("Last Saturday: "+lastdate) #<=

def recrusive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
folder_write = '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Investigation_of_differnt_versions/output_upto20190330_Decile_MergeEmail_Remove_'+str(datetime.datetime.now().date())+'/'
try:
    os.stat(folder_write)
except:
    os.mkdir(folder_write)
    
os.getcwd()

Lapsed Start on: 2017-10-01
Active Start on: 2018-04-01
Store Allocation Starting on: 2017-10-01
Last Saturday: 2019-03-30


'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Investigation_of_differnt_versions'

In [2]:
chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_time',
                   'transaction_id','location_id','total_transaction_amt'],nrows=samplerows): #Add back the transaction info
    chunk['total_transaction_amt']=chunk['total_transaction_amt'].astype(float)
    chunk = chunk.drop_duplicates()
    
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())


del chunk
print("Earliest Date:" + str(dftrans_before_20180922['transaction_date'].min()))

dftrans_before_20180922=dftrans_before_20180922.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
dftrans_before_20180922=dftrans_before_20180922.drop_duplicates("customer_id_hashed")

print("Deduped",datetime.datetime.now())


logging.info("Deduped: "+str(datetime.datetime.now()))
del dftrans_before_20180922['transaction_time']

gc.collect()


1 2019-04-18 10:25:41.701321
2 2019-04-18 10:26:22.846903
3 2019-04-18 10:26:59.080279
4 2019-04-18 10:27:40.476683
5 2019-04-18 10:28:22.466453
6 2019-04-18 10:29:13.805466
7 2019-04-18 10:30:06.917008
8 2019-04-18 10:30:56.661353
9 2019-04-18 10:31:53.278814
10 2019-04-18 10:32:47.684026
11 2019-04-18 10:34:02.601595
12 2019-04-18 10:35:49.971883
13 2019-04-18 10:37:40.310354
14 2019-04-18 10:40:19.548358
15 2019-04-18 10:42:02.192818
Earliest Date:2016-06-26
Deduped 2019-04-18 11:03:39.755948


84

In [3]:
# Up to 2019-03-30
# All item level data, weekly and the 1-time transfered historical data
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recrusive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,3,30)]
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>datetime.date(2018,9,22)]
print(historical_daily_df.shape)
print("Min_Date: "+str(historical_daily_df['week_end_dt'].min()))
print("Max_Date: "+str(historical_daily_df['week_end_dt'].max()))

(20, 2)
Min_Date: 2018-09-29
Max_Date: 2019-02-09


In [4]:
new_daily_data_folder="/home/jian/BigLots/2019_by_weeks/"
new_daily_data_list=list(recrusive_file_gen(new_daily_data_folder))
new_daily_data_list=[x for x in new_daily_data_list if (".txt" in x) & ("DailySales" in x)]
new_daily_data_list=[x for x in new_daily_data_list if "hist" not in x]

new_daily_df=pd.DataFrame({"file_path":new_daily_data_list})

new_daily_df['week_end_dt']=new_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("2019_by_weeks/MediaStorm_")[1][:10])
new_daily_df['week_end_dt']=new_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
new_daily_df=new_daily_df[new_daily_df['week_end_dt']>historical_daily_df['week_end_dt'].max()]
new_daily_df=new_daily_df[new_daily_df['week_end_dt']<=datetime.date(2019,3,30)]
print(new_daily_df.shape)
print("Min_Date: "+str(new_daily_df['week_end_dt'].min()))
print("Max_Date: "+str(new_daily_df['week_end_dt'].max()))

daily_df_file_after_20180922=historical_daily_df.append(new_daily_df)
new_dailysales_files=daily_df_file_after_20180922['file_path'].tolist()

(7, 2)
Min_Date: 2019-02-16
Max_Date: 2019-03-30


In [5]:
combined_rewards_transaction_after_20180922_agg=pd.DataFrame() 
count_i=1
print("Total Files: "+str(len(new_dailysales_files)))
for file_daily in new_dailysales_files:
    df=pd.read_table(file_daily,sep= '|',dtype =str,nrows=samplerows,
                     usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df.groupby(['customer_id_hashed','transaction_dt','transaction_id','location_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df=df.drop_duplicates()

    
    combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.append(df)
    print(count_i,"done",datetime.datetime.now())
    count_i+=1
del df
gc.collect()
combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.rename(columns={"transaction_dt":"transaction_date"})


Total Files: 27
1 done 2019-04-18 11:05:16.877621
2 done 2019-04-18 11:06:11.960628
3 done 2019-04-18 11:06:54.733328
4 done 2019-04-18 11:07:38.984404
5 done 2019-04-18 11:08:26.194611
6 done 2019-04-18 11:09:15.877165
7 done 2019-04-18 11:10:09.378583
8 done 2019-04-18 11:11:10.817855
9 done 2019-04-18 11:12:34.454194
10 done 2019-04-18 11:13:57.131669
11 done 2019-04-18 11:15:28.239436
12 done 2019-04-18 11:17:10.125789
13 done 2019-04-18 11:19:04.433835
14 done 2019-04-18 11:20:25.728640
15 done 2019-04-18 11:21:28.107393
16 done 2019-04-18 11:22:29.820762
17 done 2019-04-18 11:23:43.350256
18 done 2019-04-18 11:24:52.671074
19 done 2019-04-18 11:25:52.723832
20 done 2019-04-18 11:26:55.690475
21 done 2019-04-18 11:28:00.009239
22 done 2019-04-18 11:29:05.379335
23 done 2019-04-18 11:30:15.974040
24 done 2019-04-18 11:31:26.693073
25 done 2019-04-18 11:32:35.221665
26 done 2019-04-18 11:33:45.232130
27 done 2019-04-18 11:34:58.946155


In [6]:
gc.collect()

combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.groupby(['customer_id_hashed','transaction_date','transaction_id','location_id'])['item_transaction_amt'].sum().to_frame().reset_index()
combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.rename(columns={"item_transaction_amt":"total_transaction_amt"})

all_rewards_since_201606=dftrans_before_20180922.append(combined_rewards_transaction_after_20180922_agg)

del dftrans_before_20180922
del combined_rewards_transaction_after_20180922_agg
gc.collect()

42

In [7]:
all_rewards_since_201606.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Investigation_of_differnt_versions/BL_Rewards_Transactions_20160626_to_20190330.csv")
gc.collect()

0

In [8]:
'''
# Lapsed data

lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str) # Doesn't go to score at all, so no need to read all columns

print("Lapsed earliest:" + lapsed['transaction_date'].min())
print("Lapsed newest:" + lapsed['transaction_date'].max())
'''

'\n# Lapsed data\n\nlapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",\n                     sep=",",usecols=[\'customer_id_hashed\',\'transaction_date\'],dtype=str) # Doesn\'t go to score at all, so no need to read all columns\n\nprint("Lapsed earliest:" + lapsed[\'transaction_date\'].min())\nprint("Lapsed newest:" + lapsed[\'transaction_date\'].max())\n'

In [9]:
#Getting the store for an id

frequently_visit_stores_18_months=all_rewards_since_201606[all_rewards_since_201606['transaction_date']>=Beginning_18_months_ago]

frequently_visit_stores_2=frequently_visit_stores_18_months.groupby(['customer_id_hashed','location_id'])['total_transaction_amt'].sum().to_frame().reset_index().rename(columns={"total_transaction_amt":"sales"})
frequently_visit_stores_18_months=frequently_visit_stores_18_months.groupby(['customer_id_hashed','location_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})

frequently_visit_stores_18_months=pd.merge(frequently_visit_stores_18_months,frequently_visit_stores_2,on=['customer_id_hashed','location_id'],how="outer")
del frequently_visit_stores_2
print(frequently_visit_stores_18_months.shape)
frequently_visit_stores_18_months=frequently_visit_stores_18_months.sort_values(['customer_id_hashed','trans','sales'],ascending=[True,False,False])

frequently_visit_stores_18_months=frequently_visit_stores_18_months[['customer_id_hashed','location_id']].drop_duplicates("customer_id_hashed")
print(frequently_visit_stores_18_months.shape)
frequently_visit_stores_18_months.to_csv(folder_write+"frequently_visit_stores_18_months.csv",index=False)
gc.collect()

(24905063, 4)
(20569010, 2)


0

In [10]:
###get recency
dfrecency=all_rewards_since_201606[['customer_id_hashed','transaction_date']].sort_values("transaction_date",ascending=False).drop_duplicates()#Allready combined

print (min(dfrecency['transaction_date']))
print (max(dfrecency['transaction_date']))
dfrecency = dfrecency.drop_duplicates('customer_id_hashed')
dfrecency.to_csv(folder_write + 'dfrecency.csv',index = False)

2016-06-26
2019-03-30


In [11]:
dfrecency['transaction_date'] = pd.to_datetime(dfrecency['transaction_date'])
dfrecency['recency'] =  datetime.datetime.strptime(str(lastdate), '%Y-%m-%d').date() - dfrecency['transaction_date']
dfrecency['recency'] = dfrecency['recency'].apply(lambda x:x.days)
dfrecency['recency'] = np.ceil((dfrecency['recency']+1)/30)

dfrecency = dfrecency[['customer_id_hashed','recency']]
dfrecency = dfrecency.drop_duplicates('customer_id_hashed')
dfrecency.to_csv(folder_write + 'dfrecency2.csv',index = False)

dfrecency.shape

(25770895, 2)

In [12]:
dfiddetail = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',nrows = samplerows)
dfiddetail = dfiddetail.drop_duplicates('customer_id_hashed')
#########
new_sign_ups_2019_list=list(recrusive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
new_sign_ups_2019_list=sorted([x for x in new_sign_ups_2019_list if "ster" in x])

new_sign_ups_2019_df=pd.DataFrame({"file_path":new_sign_ups_2019_list})
new_sign_ups_2019_df['Date']=new_sign_ups_2019_df['file_path'].apply(lambda x: x.split("MediaStorm_")[1][:10])
new_sign_ups_2019_df=new_sign_ups_2019_df[new_sign_ups_2019_df['Date']<=lastdate]

for file_new_signups in new_sign_ups_2019_df['file_path'].tolist():
    df=pd.read_table(file_new_signups,dtype=str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'],sep="|",nrows=samplerows)
    dfiddetail=df.append(dfiddetail) # Already sorted and newest kept on the top
    print(datetime.datetime.now(),file_new_signups)
dfiddetail=dfiddetail.drop_duplicates("customer_id_hashed")

######
dfiddetail2 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     nrows = samplerows,dtype = 'str',sep = '|',
                       usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'])
dfiddetail = dfiddetail.append(dfiddetail2,ignore_index = True)
dfiddetail = dfiddetail.drop_duplicates('customer_id_hashed')
dfiddetail = dfiddetail.drop_duplicates('email_address_hash')

del dfiddetail2
del df
gc.collect()

2019-04-18 12:25:59.447855 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-01-12/MediaStormMasterBiWeekly20190115-132855-055.txt
2019-04-18 12:26:06.287462 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-01-26/MediaStormMasterBiWeekly20190129-130902-016.txt
2019-04-18 12:26:12.048389 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-02/MediaStormMasterWeekly20190205-111610-675.txt
2019-04-18 12:26:21.199527 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-09/MediaStormMasterWeekly20190212-122428-267.txt
2019-04-18 12:26:27.664936 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormMasterWeekly20190219-113650-867.txt
2019-04-18 12:26:35.004173 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormMasterWeekly20190226-112921-061.txt
2019-04-18 12:26:41.333909 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormMasterWeekly20190305-112945-302.txt
2019-04-18 12:26:48.209184 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormMasterWe

150

In [13]:
all_rewards_since_201606['transactions'] = 1
dftotal = all_rewards_since_201606[['customer_id_hashed','total_transaction_amt','transactions']].groupby(['customer_id_hashed']).sum().reset_index().rename(columns={"total_transaction_amt":"sales"})

dftotal = pd.merge(dftotal,dfrecency,on = 'customer_id_hashed',how='outer')
del dfrecency

gc.collect()

49

In [14]:
dftotal.head(2)

customer_id_hashed   sales  transactions  \
0  00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...  244.58             4   
1  000001dadc0265bf9d250566d74e0006323f18b5826641...  143.25             3   

   recency  
0      6.0  
1      4.0

In [15]:
dftotal = dftotal.sort_values(['transactions','recency','sales'],ascending = [0,1,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'Transindex'})

dftotal = dftotal.sort_values(['sales','recency','transactions'],ascending = [0,1,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'Amtindex'})

dftotal = dftotal.sort_values(['recency','transactions','sales'],ascending = [1,0,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'recencyindex'})

c_ids = len(dftotal.index)
logging.info('total customers from transaction and amt: ')
logging.info(c_ids)
c_ids = np.ceil(c_ids/5.0)

dftotal['Transindex'] = np.ceil((dftotal['Transindex']+1)/c_ids)
dftotal['Amtindex'] = np.ceil((dftotal['Amtindex']+1)/c_ids)
dftotal['recencyindex'] = np.ceil((dftotal['recencyindex']+1)/c_ids)

dftotal['RFM'] = dftotal['recencyindex']*100 + dftotal['Transindex']*10 + dftotal['Amtindex']
dftotal = dftotal.sort_values(['RFM','recency','transactions',
                               'sales'],ascending = [1,1,0,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'frmindex'})
c_ids = len(dftotal.index)
c_ids = np.ceil(c_ids/10.0)
dftotal['frmindex'] = np.ceil((dftotal['frmindex']+1)/c_ids)

dftotal.to_csv(folder_write + 'dfrfm.csv',index = False)

dftotal = pd.read_csv(folder_write + 'dfrfm.csv')

dftotal = dftotal[['customer_id_hashed','frmindex']]

dfrecency = pd.read_csv(folder_write + 'dfrecency.csv')
dfrecency['active'] = np.where(dfrecency['transaction_date']>=active_Sunday,'active',
                               np.where(dfrecency['transaction_date']>=lapsed_Sunday,'lapsed','other')
                              )

print(dfrecency['active'].unique().tolist())

In [21]:
dftotal = pd.merge(dftotal,dfrecency[['customer_id_hashed','active']],on = 'customer_id_hashed')

dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].astype('str')
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].str[0:5]
dfiddetail['customer_zip_code'].fillna('00000',inplace = True)
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].apply(lambda x:x.zfill(5))
print(len(dfiddetail.index))


# In[18]:


print("totalids_trans:",len(dftotal.index))
dftotal = pd.merge(dftotal,dfiddetail,on = 'customer_id_hashed')
print("totalids_trans_mergewithmaster:",len(dftotal.index))


# In[20]:


zipmap = pd.read_csv('/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/zip_with_ta_dma.csv',dtype = 'str')
zipmap['zipcodegroup'] = zipmap['revenue_flag']
zipmap = zipmap[['zip','zipcodegroup']].drop_duplicates('zip')
zipmap.columns = ['customer_zip_code','zipcodegroup']
dftotal = pd.merge(dftotal,zipmap,on ='customer_zip_code',how = 'left' )
print(dftotal['zipcodegroup'].unique())
dftotal['zipcodegroup'].fillna('T',inplace = True)
print(dftotal['zipcodegroup'].unique())


# In[21]:

dftotal.to_csv(folder_write + 'dfrfm_wemail.csv',index = False)
print("Final wemailcsv:",dftotal.shape)

24959949
totalids_trans: 25770895
totalids_trans_mergewithmaster: 24077571
['P' 'S' nan 'T']
['P' 'S' 'T']
Final wemailcsv: (24077571, 6)


In [22]:
# Remove the ids don't have transactions within 18 months
df_other_18_plus=dftotal[dftotal['active']=="other"][['customer_id_hashed','email_address_hash']]
df_other_18_plus['segment']="18_months_plus_back_201606"

dftotal=dftotal[dftotal['active']!="other"]



# Getting the primary stores for each member

In [23]:
# frequently_visit_stores_18_months=pd.read_csv(folder_write+"frequently_visit_stores_18_months.csv",dtype=str)
register_stores=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/output_sing_up_location/BL_id_by_register_store_JL_2019-04-09.csv",
                            dtype=str,nrows=samplerows)
register_stores=register_stores[['customer_id_hashed','sign_up_location']].rename(columns={"sign_up_location":"location_id"})




In [24]:
store_for_ids=frequently_visit_stores_18_months.append(register_stores)
store_for_ids=store_for_ids.drop_duplicates("customer_id_hashed")
store_for_ids.shape

(26754444, 2)

In [25]:
del frequently_visit_stores_18_months
del register_stores
gc.collect()

25

In [26]:
dftotal=pd.merge(dftotal,store_for_ids,on="customer_id_hashed",how="left")

In [27]:
# read the quadrant by store for 2018 Q4

Q4_store_quadrant=pd.read_excel("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Excel_BL_2018_Q4_post_YoY_small_JL_2019-03-04.xlsx",
                                dtype=str,sheetname="Q4_Store_Quadrant_Defination",usecols=['location_id','Quadrant'])
print(Q4_store_quadrant.shape)
print(len(Q4_store_quadrant['location_id'].unique()))

dftotal=pd.merge(dftotal,Q4_store_quadrant,on="location_id",how="left")
dftotal['Quadrant']=dftotal['Quadrant'].fillna("Quadrant III")


(1361, 2)
1361


In [28]:
gc.collect()

89172

# Summary

In [29]:
dftotal['frmindex']=dftotal['frmindex'].apply(lambda x: str(int(float(x))).zfill(2))
dftotal['customer_zip_code']=dftotal['customer_zip_code'].apply(lambda x: x.zfill(5))
dftotal['frmindex']=dftotal['frmindex'].apply(lambda x:"D"+x)

In [30]:
dftotal.to_csv(folder_write + 'dfrfm_final_details_wemail_zip_StoreQuad.csv',index = False)
gc.collect()

0

In [31]:
dftotal['HML_Group']=np.where(dftotal['frmindex'].isin(['D01','D02','D03','D04']),"H",
                                        np.where(dftotal['frmindex'].isin(['D05','D06','D07']),"M","L"))

dftotal['segment_2019Q2']=dftotal['Quadrant']+"_"+dftotal['zipcodegroup']+"_"+dftotal['HML_Group']+"_2019Q2"


In [32]:

import random
random.seed(1)
total_rows=len(dftotal)

test_all_df=pd.DataFrame()
control_all_df=pd.DataFrame()

i_counter=0

for seg,group in dftotal.groupby(['segment_2019Q2']):
    random_list=random.sample(range(len(group)), int(np.round(len(group)/total_rows*500000)))

    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    df_control=group[group['index'].isin(random_list)]
    df_test=group[~group['index'].isin(random_list)]
    
    df_control['segment_2019Q2']="C_"+df_control['segment_2019Q2']
    df_test['segment_2019Q2']="T_"+df_test['segment_2019Q2']
    test_all_df=test_all_df.append(df_test)
    control_all_df=control_all_df.append(df_control)
    i_counter+=1
    print(i_counter,datetime.datetime.now())


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 2019-04-18 13:11:55.288597
2 2019-04-18 13:11:58.887805
3 2019-04-18 13:12:03.737250
4 2019-04-18 13:12:08.171955
5 2019-04-18 13:12:11.833088
6 2019-04-18 13:12:15.843128
7 2019-04-18 13:12:19.957492
8 2019-04-18 13:12:23.754446
9 2019-04-18 13:12:27.681960
10 2019-04-18 13:12:35.053241
11 2019-04-18 13:12:41.786924
12 2019-04-18 13:12:49.556440
13 2019-04-18 13:12:57.326375
14 2019-04-18 13:13:04.191628
15 2019-04-18 13:13:11.315941
16 2019-04-18 13:13:18.576994
17 2019-04-18 13:13:25.249444
18 2019-04-18 13:13:32.358540
19 2019-04-18 13:13:43.361885
20 2019-04-18 13:13:53.983144
21 2019-04-18 13:14:05.906142
22 2019-04-18 13:14:17.492290
23 2019-04-18 13:14:28.145348
24 2019-04-18 13:14:38.892505
25 2019-04-18 13:14:50.089314
26 2019-04-18 13:15:01.040312
27 2019-04-18 13:15:12.187270
28 2019-04-18 13:15:35.940502
29 2019-04-18 13:15:56.428978
30 2019-04-18 13:16:26.128334
31 2019-04-18 13:16:50.483983
32 2019-04-18 13:17:14.140514
33 2019-04-18 13:17:36.346531
34 2019-04-18 13:18

In [33]:
del dftotal
gc.collect()

20

In [34]:
folder_write

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Investigation_of_differnt_versions/output_upto20190330_Decile_MergeEmail_Remove_2019-04-18/'

In [35]:
test_all_df.to_csv(folder_write+"all_test.csv",index=False)
control_all_df.to_csv(folder_write+"all_control.csv",index=False)

folder_write_inner = folder_write+'by_group/'
try:
    os.stat(folder_write_inner)
except:
    os.mkdir(folder_write_inner)

In [36]:
i_counter=0
for seg,group in test_all_df.groupby(['segment_2019Q2']):
    group=group[['customer_id_hashed','email_address_hash','segment_2019Q2']].rename(columns={"segment_2019Q2":"segment"})
    group.to_csv(folder_write_inner+seg+".csv",index=False)
    i_counter+=1
    print(i_counter,seg,datetime.datetime.now())

1 T_Quadrant III_P_H_2019Q2 2019-04-18 13:22:40.027737
2 T_Quadrant III_P_L_2019Q2 2019-04-18 13:22:43.000501
3 T_Quadrant III_P_M_2019Q2 2019-04-18 13:22:49.364981
4 T_Quadrant III_S_H_2019Q2 2019-04-18 13:22:51.699709
5 T_Quadrant III_S_L_2019Q2 2019-04-18 13:22:52.430964
6 T_Quadrant III_S_M_2019Q2 2019-04-18 13:22:54.220080
7 T_Quadrant III_T_H_2019Q2 2019-04-18 13:22:55.289936
8 T_Quadrant III_T_L_2019Q2 2019-04-18 13:22:55.656271
9 T_Quadrant III_T_M_2019Q2 2019-04-18 13:22:56.645103
10 T_Quadrant II_P_H_2019Q2 2019-04-18 13:23:04.724115
11 T_Quadrant II_P_L_2019Q2 2019-04-18 13:23:07.106268
12 T_Quadrant II_P_M_2019Q2 2019-04-18 13:23:12.280964
13 T_Quadrant II_S_H_2019Q2 2019-04-18 13:23:13.991405
14 T_Quadrant II_S_L_2019Q2 2019-04-18 13:23:14.457246
15 T_Quadrant II_S_M_2019Q2 2019-04-18 13:23:15.643045
16 T_Quadrant II_T_H_2019Q2 2019-04-18 13:23:16.420959
17 T_Quadrant II_T_L_2019Q2 2019-04-18 13:23:16.654190
18 T_Quadrant II_T_M_2019Q2 2019-04-18 13:23:17.260092
19 T_Quadr

In [37]:
i_counter=0
for seg,group in control_all_df.groupby(['segment_2019Q2']):
    group=group[['customer_id_hashed','email_address_hash','segment_2019Q2']].rename(columns={"segment_2019Q2":"segment"})
    group.to_csv(folder_write_inner+seg+".csv",index=False)
    i_counter+=1
    print(i_counter,seg,datetime.datetime.now())

1 C_Quadrant III_P_H_2019Q2 2019-04-18 13:24:55.912765
2 C_Quadrant III_P_L_2019Q2 2019-04-18 13:24:55.984570
3 C_Quadrant III_P_M_2019Q2 2019-04-18 13:24:56.144131
4 C_Quadrant III_S_H_2019Q2 2019-04-18 13:24:56.197887
5 C_Quadrant III_S_L_2019Q2 2019-04-18 13:24:56.216406
6 C_Quadrant III_S_M_2019Q2 2019-04-18 13:24:56.258639
7 C_Quadrant III_T_H_2019Q2 2019-04-18 13:24:56.283318
8 C_Quadrant III_T_L_2019Q2 2019-04-18 13:24:56.294117
9 C_Quadrant III_T_M_2019Q2 2019-04-18 13:24:56.317809
10 C_Quadrant II_P_H_2019Q2 2019-04-18 13:24:56.515946
11 C_Quadrant II_P_L_2019Q2 2019-04-18 13:24:56.573293
12 C_Quadrant II_P_M_2019Q2 2019-04-18 13:24:56.703064
13 C_Quadrant II_S_H_2019Q2 2019-04-18 13:24:56.744206
14 C_Quadrant II_S_L_2019Q2 2019-04-18 13:24:56.757557
15 C_Quadrant II_S_M_2019Q2 2019-04-18 13:24:56.786794
16 C_Quadrant II_T_H_2019Q2 2019-04-18 13:24:56.806610
17 C_Quadrant II_T_L_2019Q2 2019-04-18 13:24:56.815051
18 C_Quadrant II_T_M_2019Q2 2019-04-18 13:24:56.831517
19 C_Quadr

In [49]:
lapsed_trans=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed'],dtype=str).drop_duplicates() # Doesn't go to score at all, so no need to read all columns
lapsed_trans['lapsed_trans']=True

lapsed_master=pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     nrows = samplerows,dtype = 'str',sep = '|',
                       usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'])

lapsed_master=lapsed_master.drop_duplicates("customer_id_hashed")
print(lapsed_master.shape)

lapsed_master=pd.merge(lapsed_master,lapsed_trans,on="customer_id_hashed",how="outer")
print(lapsed_master.shape)
lapsed_master=lapsed_master[~pd.isnull(lapsed_master['email_address_hash'])]

# remove the non-match email ids at the end and no calculation for the WD here

(1873409, 3)
(1873954, 4)


In [50]:
summary_18_plus_back_20160626=df_other_18_plus.groupby('segment')['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})


In [51]:
lapsed_master=lapsed_master[~lapsed_master['customer_id_hashed'].isin(test_all_df['customer_id_hashed'])]
lapsed_master=lapsed_master[~lapsed_master['customer_id_hashed'].isin(control_all_df['customer_id_hashed'])]
lapsed_master=lapsed_master[~lapsed_master['email_address_hash'].isin(test_all_df['email_address_hash'])]
lapsed_master=lapsed_master[~lapsed_master['email_address_hash'].isin(control_all_df['email_address_hash'])]

lapsed_master=lapsed_master[~lapsed_master['email_address_hash'].isin(df_other_18_plus['email_address_hash'])]
lapsed_master=lapsed_master[~lapsed_master['email_address_hash'].isin(df_other_18_plus['email_address_hash'])]
lapsed_master=lapsed_master[~lapsed_master['email_address_hash'].isin(df_other_18_plus['email_address_hash'])]
lapsed_master=lapsed_master[~lapsed_master['email_address_hash'].isin(df_other_18_plus['email_address_hash'])]

lapsed_master.shape

(832243, 4)

In [52]:
lapsed_master=lapsed_master[['customer_id_hashed','email_address_hash']]
lapsed_master['segment']="WalkingDead_2019Q2"

In [53]:
lapsed_master.to_csv(folder_write_inner+"WalkingDead_Group_before_20160626.csv",index=False)

In [54]:
summary_test=test_all_df.groupby('segment_2019Q2')['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count","segment_2019Q2":"segment"})
summary_control=control_all_df.groupby('segment_2019Q2')['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count","segment_2019Q2":"segment"})
summary_WD=lapsed_master.groupby('segment')['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})

summary_overll=summary_test.append(summary_control).append(summary_WD).append(summary_18_plus_back_20160626)


summary_overll.to_csv(folder_write_inner+"test_control_groups_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [55]:
summary_overll

segment  id_count
0    T_Quadrant III_P_H_2019Q2   1116280
1    T_Quadrant III_P_L_2019Q2    338861
2    T_Quadrant III_P_M_2019Q2    846723
3    T_Quadrant III_S_H_2019Q2    250381
4    T_Quadrant III_S_L_2019Q2     78687
5    T_Quadrant III_S_M_2019Q2    217277
6    T_Quadrant III_T_H_2019Q2    115058
7    T_Quadrant III_T_L_2019Q2     41947
8    T_Quadrant III_T_M_2019Q2    115850
9     T_Quadrant II_P_H_2019Q2   1032149
10    T_Quadrant II_P_L_2019Q2    266506
11    T_Quadrant II_P_M_2019Q2    694612
12    T_Quadrant II_S_H_2019Q2    186962
13    T_Quadrant II_S_L_2019Q2     52865
14    T_Quadrant II_S_M_2019Q2    146561
15    T_Quadrant II_T_H_2019Q2     89691
16    T_Quadrant II_T_L_2019Q2     29514
17    T_Quadrant II_T_M_2019Q2     76154
18    T_Quadrant IV_P_H_2019Q2   1193125
19    T_Quadrant IV_P_L_2019Q2    336533
20    T_Quadrant IV_P_M_2019Q2    822073
21    T_Quadrant IV_S_H_2019Q2    285578
22    T_Quadrant IV_S_L_2019Q2     85396
23    T_Quadrant IV_S_M_2019Q2    226863
24    T_Quadrant IV_T_H_2019Q2    128402
25    T_Quadrant IV_T_L_2019Q2     45417
26    T_Quadrant IV_T_M_2019Q2    115949
27     T_Quadrant I_P_H_2019Q2   3708311
28     T_Quadrant I_P_L_2019Q2    895825
29     T_Quadrant I_P_M_2019Q2   2439725
..                         ...       ...
8    C_Quadrant III_T_M_2019Q2      3152
9     C_Quadrant II_P_H_2019Q2     28082
10    C_Quadrant II_P_L_2019Q2      7251
11    C_Quadrant II_P_M_2019Q2     18899
12    C_Quadrant II_S_H_2019Q2      5087
13    C_Quadrant II_S_L_2019Q2      1438
14    C_Quadrant II_S_M_2019Q2      3988
15    C_Quadrant II_T_H_2019Q2      2440
16    C_Quadrant II_T_L_2019Q2       803
17    C_Quadrant II_T_M_2019Q2      2072
18    C_Quadrant IV_P_H_2019Q2     32462
19    C_Quadrant IV_P_L_2019Q2      9156
20    C_Quadrant IV_P_M_2019Q2     22367
21    C_Quadrant IV_S_H_2019Q2      7770
22    C_Quadrant IV_S_L_2019Q2      2323
23    C_Quadrant IV_S_M_2019Q2      6172
24    C_Quadrant IV_T_H_2019Q2      3494
25    C_Quadrant IV_T_L_2019Q2      1236
26    C_Quadrant IV_T_M_2019Q2      3155
27     C_Quadrant I_P_H_2019Q2    100894
28     C_Quadrant I_P_L_2019Q2     24373
29     C_Quadrant I_P_M_2019Q2     66379
30     C_Quadrant I_S_H_2019Q2     21772
31     C_Quadrant I_S_L_2019Q2      5818
32     C_Quadrant I_S_M_2019Q2     16922
33     C_Quadrant I_T_H_2019Q2      9493
34     C_Quadrant I_T_L_2019Q2      2981
35     C_Quadrant I_T_M_2019Q2      8259
0           WalkingDead_2019Q2    832243
0   18_months_plus_back_201606   5200288

[74 rows x 2 columns]